In [1]:
push!(LOAD_PATH, "/Users/jayyao/Documents/Research/TraceFormula/Module")
using GridapEM
using Gridap
using DelimitedFiles
using KrylovKit
using LinearAlgebra
using GridapMakie
using GLMakie

┌ Info: Precompiling GridapEM [top-level]
└ @ Base loading.jl:1342


In [25]:
run(`scp wyao@txe1-login.mit.edu:/home/gridsan/wyao/Research/TraceFormula/AveragedSE/r0.9/task1/geometry.msh /Users/jayyao/Documents/Research/TraceFormula/AveragedSE/`)

Process(`scp wyao@txe1-login.mit.edu:/home/gridsan/wyao/Research/TraceFormula/AveragedSE/r0.9/task1/geometry.msh /Users/jayyao/Documents/Research/TraceFormula/AveragedSE/`, ProcessExited(0))

In [26]:
"""This part is used to define all parameters used"""
# Geometry parameters of the mesh
# Rectangular with center circle design domain
λ = 1.0           # Wavelength
L = 2.5           # Width of the rectangular domain
H = 2.5           # Height of the rectangular domain
rd = 0.9          # Radius of the design domain circle
rt = rd + 0.2     # Radius of the target circle
dpml = 0.5        # Thickness of PML

# Characteristic length (controls the resolution, smaller the finer)
resol = 40.0      # Number of points per wavelength
l1 = λ / resol    # Normal region
l2 = l1 / 2.0     # Design region
l3 = 2 * l1       # PML

# Physical parameters 
k = 2 * π / λ       # Wave number 
# Bloch wavevector
kb = VectorValue(0.0, 0.0)     
ω = k               # c=1
ϵ1 = 1.0            # Relative electric permittivity for material 1 (y > 0)
ϵ2 = 1.0            # Relative electric permittivity for material 2 (y < 0)
ϵ3 = 0.0            # Relative electric permittivity for potential waveguide
ϵd = 12.0           # Relative electric permittivity for design material
μ = 1.0             # Relative magnetic permeability for all materials
wg_center = [0, 0]  # Waveguide center if exist
wg_size = [0, 0]    # Waveguide size if exist
#LHp = [Inf, h1 + h2]
#LHn = [Inf, h3]

# PML parameters
R = 1e-10           # Tolerence for PML reflection 
σ1 = -3 / 4 * log(R) / dpml / √ϵ1
σ2 = -3 / 4 * log(R) / dpml / √ϵ2
σs = [σ1, σ2]
############  Optimization parameters #############
flag_f = true       # Turn on filter
flag_t = true       # Turn on threshold

# Filter and threshold paramters
r = [0.02 * λ, 0.02 * λ]  # Filter radius
β = 80.0                  # β∈[1,∞], threshold sharpness
η = 0.5                   # η∈[0,1], threshold center

α = 1.0 / (2 * 1000.0)    # Equivalent loss α = 1/2Q

# Number of subspace
K = 20

# Amplify g for NLopt
Amp = 1

# Sum over kx
nkx = 30
nparts = nkx / 2

Bρ = true          # Matrix B depend on parameters?
ρv = 0.5

# Foundary constraint parameters
c = 0#resol^4
lw = r[1]
ls = r[1]
ηe = fηe(lw / r[1])
ηd = fηd(lw / r[1])


# Create mesh file
geo_param = CirRecGeometry(L, H, rd, rt, dpml, l1, l2, l3)
#geo_param = PeriodicGeometry(L, h1, h2, h3, ht, hs, dpml, l1, l2, l3)
meshfile_name = "geometry.msh"
#MeshGenerator(geo_param, meshfile_name)

# Apply gridap finite element analysis to mesh file
gridap = GridapFE(meshfile_name, 1, 2, ["DirichletEdges", "DirichletNodes"], ["DesignNodes", "DesignEdges"], ["Target"], [], flag_f)
#run(`gmsh geometry.msh`)

# Change default physics parameters
LHp=[L / 2, H / 2]  # Start of PML for x,y > 0
LHn=[L / 2, H / 2]  # Start of PML for x,y < 0

phys = PhysicalParameters(k, kb, ω, ϵ1, ϵ2, ϵ3, ϵd, μ, R, σs, dpml, LHp, LHn, wg_center, wg_size)

control = ControllingParameters(flag_f, flag_t, r, β, η, α, nparts, nkx, K, Amp, Bρ, ρv, c, ηe, ηd)

# ρ_circ(x, r) = (x[1]^2 + x[2]^2) < r^2 ? 1 : 0
# r_init = (rd - rd / sqrt(2)) * 1 + rd / sqrt(2)
# lc_temp(v) = ∫(v * x->ρ_circ(x, r_init))gridap.dΩ
# ρc_vec = assemble_vector(lc_temp, gridap.FE_P)
# ρ_init = ρ_extract(ρc_vec; gridap)
# #ρ_init[ρ_init .< 0.5] .= 0
# ρ_init[ρ_init .> 0] .= 0.5
# @show sum(ρ_init) / gridap.np, maximum(ρ_init)


Info    : Reading 'geometry.msh'...
Info    : 60 entities
Info    : 34254 nodes
Info    : 69252 elements
Info    : Done reading 'geometry.msh'


ControllingParameters(true, true, [0.02, 0.02], 80.0, 0.5, 0.0005, 15, 30, 20, 1.0, true, 0.5, 0.0, 0.75, 0.25)

In [27]:
# Output and view
for task = 3 : 3
ImportFromFile = true
g_opt = readdlm("r$(rd)/task$(task)/g_opt_value.txt", Float64)

N = num_free_dofs(gridap.FE_U)
if ImportFromFile
    ρW_max = readdlm("r$(rd)/task$(task)/pW_opt_value.txt", Float64)
    ρW_max = ρW_max[:]
    ρ0 = ρW_max[1 : gridap.np]
    W_mat = reinterpret(ComplexF64, reshape(ρW_max[gridap.np + 1 : end], (2 * N, control.K)))
else
    ρ0 = ones(gridap.np)
end
 
ρf_vec = ρf_ρ0(ρ0; control, gridap)
ρfh = FEFunction(gridap.FE_Pf, ρf_vec)
ρth = (ρf -> Threshold(ρf; control)) ∘ ρfh
        
A_mat = MatrixA(ρth; phys, control, gridap)
B_mat = MatrixB(ρth; control, gridap)
@show sum(∫(ρth)gridap.dΩ_d) / sum(∫(1)gridap.dΩ_d)

A0_mat = MatrixA0(phys, control, gridap)
O_mat = MatrixOc(phys.k, phys.ϵ1; gridap)

G_ii, V_raw, info = eigsolve(x -> A_mat \ (A0_mat * x), rand(ComplexF64, N), 10, :LM)
G_trace, W_raw, info = eigsolve(x -> MatrixG(x; A_mat, B_mat, O_mat), rand(ComplexF64, N), min(control.K, 10), :LM; krylovdim = 30)
if !ImportFromFile
    W_mat = rand(ComplexF64, N, control.K)
    for ib = 1 : control.K
        W_mat[:, ib] = W_raw[ib]
    end
end

V_mat = rand(ComplexF64, N, control.K)
for ib = 1 : min(control.K, 10)
    V_mat[:, ib] = V_raw[ib]
end
U_mat = A_mat \ (B_mat * W_mat)
uh_eig = FEFunction(gridap.FE_U, V_mat[:, 1])
uh = FEFunction(gridap.FE_U, U_mat[:, 1])

fig, ax, plt = plot(gridap.Ω, real(1 + 11 * ρth), colormap = :binary)
Colorbar(fig[1,2], plt)
ax.aspect = AxisAspect(L/(H))
ax.title = "ε"
limits!(ax, -L/2, L/2, -phys.LHn[2], phys.LHp[2])
save("Results/shape_r$(rd)_$(task)_$(Int(floor(g_opt[end]))).png", fig)
end

sum(∫(ρth) * gridap.dΩ_d) / sum(∫(1) * gridap.dΩ_d) = 0.4672965666922608


143